In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, concatenate, LSTM, Dense, Dropout, Flatten, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import numpy as np
import os
from scipy.fft import rfft # 실수 푸리에 변환 (FFT)
import pywt # 웨이블릿 변환 라이브러리
from nptdms import TdmsFile # TDMS 파일 읽기 라이브러리
from sklearn.preprocessing import MinMaxScaler, StandardScaler # 데이터 스케일링
from tensorflow.keras.optimizers import Adam # Adam 옵티마이저
import joblib # 모델 또는 스케일러 저장/로드

import pandas as pd
from sklearn.linear_model import LinearRegression # 선형 회귀

from tensorflow.keras import regularizers # 정규화 (L1, L2 등)
from tqdm import tqdm

from tensorflow.keras.models import load_model

## 데이터 전처리

In [ ]:
# TDMS 파일 로드 함수
def load_tdms_file(file_path):
    tdms_file = TdmsFile.read(file_path) # TDMS 파일 읽기

    # TDMS 파일의 첫 번째 그룹은 진동 데이터, 두 번째 그룹은 Operation 데이터
    group_name_vibration = tdms_file.groups()[0].name
    group_name_operation = tdms_file.groups()[1].name

    # 진동 데이터 채널에서 데이터 추출
    vib_channels = tdms_file[group_name_vibration].channels()
    vib_data = {ch.name.strip(): ch.data for ch in vib_channels} # 채널 이름에서 공백 제거 후 데이터 저장

    # Operation 데이터 채널에서 데이터 추출
    operation_channels = tdms_file[group_name_operation].channels()
    operation_data = {ch.name.strip(): ch.data for ch in operation_channels}

    return vib_data, operation_data

### operation

In [ ]:
HYPERPARAMETERS = {
    "interval_sec": 600,
    "threshold_r2": 0.85,
    "N_list": [3, 5],
    "window_size": 25600,
    "overlap": 0.5,
    "fixed_total_samples": 256000,
    "wavelet": 'db4',
    "level": 3,
    "top_k": 10,
    "sampling_rate": 25600
}

# 데이터 불러오기
def load_summary_from_tdms(train_root, target_folders):
    """
    Args:
        train_root (str): TDMS 파일이 포함된 트레인 데이터셋의 루트 경로.
        target_folders (list): TDMS 파일이 포함된 하위 폴더 이름 목록

    Returns:
        dict: 각 폴더 이름을 키로 하고 해당 Operation 데이터를 포함하는 DataFrame을 값으로 하는 딕셔너리.
    """
    # 각 train 폴더의 summary data를 저장할 딕셔너리
    summary = {}

    # 각 대상 폴더를 순회
    for folder in target_folders:
        # 각 폴더 내의 모든 .tdms 파일 정렬
        folder_path = os.path.join(train_root, folder)
        tdms_files = sorted([f for f in os.listdir(folder_path) if f.endswith(".tdms")])

        # 현재 폴더의 각 tdms 파일에서 추출한 레코드를 저장할 리스트
        records = []

        # 각 TDMS 파일을 순회하며 operation 데이터 로드
        for idx, tdms_file in enumerate(tdms_files):
            file_path = os.path.join(folder_path, tdms_file)
            # TDMS 파일에서 진동 데이터와 Operation 데이터를 로드. 여기서는 Operation 데이터만 사용.
            _, operation_data = load_tdms_file(file_path)

            # 각 파일은 10분(600초) 간격으로 측정되었다고 가정하고 시간(초) 계산
            time_sec = idx * 600

            # 현재 파일의 주요 Operation 데이터를 추출하여 레코드 딕셔너리 생성
            record = {
                "time_sec": time_sec,
                "TC SP Front[℃]": operation_data["TC SP Front[℃]"][0], # 첫 번째 값 사용
                "TC SP Rear[℃]": operation_data["TC SP Rear[℃]"][0],    # 첫 번째 값 사용
                "Torque[Nm]": operation_data["Torque[Nm]"][0],          # 첫 번째 값 사용
                "file_name": os.path.splitext(tdms_file)[0], # 파일 확장자를 제거한 파일 이름
                "folder": folder                             # 이후 merge를 위해 train 폴더명 기록
            }
            records.append(record)

        # 현재 폴더의 모든 레코드를 DataFrame으로 변환하여 summary 딕셔너리에 추가
        df = pd.DataFrame(records)
        summary[folder] = df

    return summary

# RUL(잔여 수명) 계산 및 라벨링
def check_stop_condition_met(df):
    """
    Args:
        df (pd.DataFrame): Operation 데이터가 포함된 DataFrame.

    Returns:
        bool: 중단 조건이 하나라도 만족하면 True, 그렇지 않으면 False.
    """
    # 중단 조건 1: Torque ≤ -17 Nm
    torque_stop = df["Torque[Nm]"] <= -17
    # 중단 조건 2: TC SP Front ≥ 200 or TC SP Rear ≥ 200
    temp_stop = (df["TC SP Front[℃]"] >= 200) | (df["TC SP Rear[℃]"] >= 200)

    # 두 조건 중 어느 하나라도 만족하는 데이터 포인트가 있으면 중단 조건 도달로 판단
    stop_condition_met = (torque_stop | temp_stop).any()
    return stop_condition_met

# 선형 회귀를 사용한 RUL 추출 함수
def estimate_rul_from_temp_time_based(temp_values, start_index, interval_sec, file_id):
    """
    Args:
        temp_values (np.array or list): 온도 데이터 (현재까지의 온도).
        start_index (int): 데이터의 시작 인덱스 (시간 계산을 위해).
        interval_sec (int): 각 데이터 포인트 간의 시간 간격(초).
        file_id (str): 로그 메시지에 사용될 파일ID.

    Returns:
        tuple: (예측 고장 시점(초), 잔여 수명(초)). 유효하지 않은 경우 (None, None).
    """
    n = len(temp_values)
    # 시간(X)축 생성: (시작 인덱스부터 n개) * 시간 간격
    x = np.arange(start_index, start_index + n) * interval_sec
    # 온도(Y)축
    y = np.array(temp_values)

    # scikit-learn 모델에 맞게 X 데이터 형태 변경
    x_reshaped = x.reshape(-1, 1)
    # 선형 회귀 모델 훈련
    model = LinearRegression().fit(x_reshaped, y)
    m = model.coef_[0]      # 기울기
    c = model.intercept_    # y절편

    # 온도가 증가하지 않는 경우(기울기가 0이하) RUL 예측 불가
    if m <= 0:
        print(f"[{file_id}] Warning: Temperature is not increasing (slope ≤ 0).")
        return None, None

    # 고장 진단 온도(200℃)에 도달하는 예측 시간 계산: (200 - c) / m
    estimated_failure_time_sec = (200 - c) / m
    # 현재 시간(마지막 데이터 포인트의 시간)으로부터 잔여 수명 계산
    rul_seconds = estimated_failure_time_sec - x[-1]

    # 예측 고장 시점이 현재 시간보다 이전이거나 RUL이 음수인 경우 유효하지 않은 결과
    if estimated_failure_time_sec < x[-1] or rul_seconds < 0:
        print(f"[{file_id}] Warning: Predicted failure is before current time. Invalid regression.")
        return None, None

    return estimated_failure_time_sec, rul_seconds

# Front/Rear 온도 칼럼 선택 함수
def choose_temp_column_closer_to_200(df, front_col, rear_col):
    """
    Args:
        df (pd.DataFrame): Operation 데이터가 포함된 DataFrame.
        front_col (str): 전면 온도 컬럼 이름.
        rear_col (str): 후면 온도 컬럼 이름.

    Returns:
        str: 200℃에 더 가까운 온도 컬럼의 이름.
    """
    # 마지막 측정값과 200℃ 사이의 절대 차이를 비교
    front_diff = abs(df[front_col].iloc[-1] - 200)
    rear_diff = abs(df[rear_col].iloc[-1] - 200)
    if front_diff <= rear_diff:
        return front_col
    else:
        return rear_col

# 3 points/ 5 points 선택 함수
def select_best_rul_with_temp_choice(df, estimation_targets, train_id, threshold_r2, interval_sec):
    """
    Args:
        df (pd.DataFrame): Operation 데이터가 포함된 DataFrame.
        estimation_targets (dict): 각 train_id에 대한 온도 컬럼 이름 튜플.
        train_id (str): 현재 처리 중인 트레인 ID.
        threshold_r2 (float): R² 값이 이 값 이상이어야 유효한 회귀로 간주.
        interval_sec (int): 각 데이터 포인트 간의 시간 간격(초).

    Returns:
        tuple: (예측 고장 시점(초), 잔여 수명(초), 사용된 데이터 포인트 수(3 또는 5)).
               유효한 예측이 없는 경우 (None, None, None).
    """
    # 중단 조건 온도에 더 가까운 온도 칼럼 선택
    front_col, rear_col = estimation_targets[train_id]
    target_col = choose_temp_column_closer_to_200(df, front_col, rear_col)

    # 각 N(3 또는 5)에 대한 RUL 추정 결과를 저장할 리스트 (R², est_time, rul, N)
    results = []

    # 3 points, 5 points를 사용하여 RUL 추정
    for n in [3, 5]:
        # 마지막 N개의 온도 값과 해당 시간 인덱스를 가져옴
        temp_values = df[target_col].values[-n:]
        start_index = len(df) - n
        x = np.arange(start_index, start_index + n) * interval_sec
        y = np.array(temp_values)

        # 선형 회귀 모델을 훈련하고 R²값 계산
        model = LinearRegression().fit(x.reshape(-1, 1), y)
        r2 = model.score(x.reshape(-1, 1), y)

        # RUL을 추정
        est_time, rul = estimate_rul_from_temp_time_based(
            temp_values=temp_values,
            start_index=start_index,
            interval_sec=interval_sec,
            file_id=f"{train_id}_{target_col}_last{n}"
        )

        # 유효한 추정 결과가 있으면 results 리스트에 추가
        if est_time is not None and rul is not None:
            results.append((r2, est_time, rul, n))

    # 우선 R²가 threshold 이상인 것 중 R²가 높은 것 선택
    filtered = [res for res in results if res[0] >= threshold_r2]

    if filtered:
        best = max(filtered, key=lambda x: x[0]) # R²가 가장 높은 결과 선택
    # 임계값을 넘는 결과가 없으면, 모든 결과 중에서 R²가 가장 높은 것을 선택
    elif results:
        best = max(results, key=lambda x: x[0])
    # 유효한 회귀 결과가 전혀 없으면 None 반환
    else:
        print(f"[{train_id}] No valid regression results.")
        return None, None, None

    r2, est_time, rul, used_n = best # 선택된 최적의 결과
    print(f"[{train_id} | {target_col}] Selected {used_n} points (R² = {r2:.4f}) → RUL = {rul:.1f} sec")
    return est_time, rul, used_n

# 중단 조건 만족하지 않은 데이터셋에 대해 RUL 라벨링
def label_rul_by_estimation(summary, estimation_targets, threshold_r2, interval_sec):
    """
    Args:
        summary (dict): 각 train_id에 대한 Operation DataFrame을 담은 딕셔너리.
        estimation_targets (dict): 회귀 예측이 필요한 train_id와 해당 온도 컬럼 튜플 딕셔너리.
        threshold_r2 (float): R² 임계값.
        interval_sec (int): 각 데이터 포인트 간의 시간 간격(초).

    Returns:
        dict: RUL이 라벨링된 DataFrame이 포함된 업데이트된 summary 딕셔너리.
    """
    # estimation_targets에 포함된 각 train_id에 대해 RUL을 라벨링
    for train_id in estimation_targets.keys():
        df = summary[train_id]

        # 1. 회귀 기반으로 RUL 추정
        est_time, rul, used_n = select_best_rul_with_temp_choice(df,estimation_targets, train_id, threshold_r2, interval_sec)

        # RUL 예측에 실패했다면 현재 train_id에 대한 라벨링을 건너뜀
        if est_time is None:
            print(f"[{train_id}] RUL 예측 실패로 라벨링하지 않음.")
            continue

        # 2. 라벨링: 원본 DataFrame을 복사하고 RUL_sec 컬럼을 추가
        df = df.copy()
        # time_sec 컬럼을 다시 계산하여 정확성을 보장
        df['time_sec'] = np.arange(len(df)) * interval_sec
        # RUL_sec 계산: 예측 고장 시점 - 현재 시점. 음수 값은 0으로 클리핑
        df['RUL_sec'] = (est_time - df['time_sec']).clip(lower=0)

        # 3. 업데이트: 원본 summary 딕셔너리에 업데이트된 DataFrame을 저장
        summary[train_id] = df

    return summary

# 중단 조건 만족 데이터셋에 대해 RUL 라벨링
def label_rul_by_stopping(summary, interval_sec):
    """
    Args:
        summary (dict): 각 train_id에 대한 Operation DataFrame을 담은 딕셔너리.
        interval_sec (int): 각 데이터 포인트 간의 시간 간격(초).

    Returns:
        dict: RUL이 라벨링된 DataFrame이 포함된 업데이트된 summary 딕셔너리.
    """
    for train_id, df in summary.items():
        df = df.copy() # 원본 DataFrame을 변경하지 않기 위해 복사

        # 조건 1: Torque ≤ -17 인덱스
        torque_stop_idx = df.index[df["Torque[Nm]"] <= -17]

        # 조건 2: TC SP Front ≥ 200 또는 TC SP Rear ≥ 200 인덱스
        temp_stop_idx = df.index[
            (df["TC SP Front[℃]"] >= 200) | (df["TC SP Rear[℃]"] >= 200)
        ]

        # 모든 중단 조건 인덱스를 합치고 가장 빠른(가장 작은) 인덱스를 찾음
        all_stop_indices = pd.concat([pd.Series(torque_stop_idx), pd.Series(temp_stop_idx)]).sort_values()

        # 중단 조건이 충족되었다면, 가장 빠른 중단 시점의 시간을 고장 시점으로 설정
        if not all_stop_indices.empty:
            failure_idx = all_stop_indices.iloc[0] # 가장 첫 번째 중단 인덱스
            failure_time_sec = df.loc[failure_idx, "time_sec"] # 해당 인덱스의 시간 값
        else:
            # 중단 조건이 충족되지 않았다면 (이 함수가 호출될 리 없지만, 만약을 위해),
            # 마지막 데이터 포인트의 시간을 고장 시점으로 가정
            failure_time_sec = df["time_sec"].max()

        # RUL 라벨링: 각 데이터 포인트의 RUL은 (고장 시점 - 현재 시점)
        # RUL이 음수가 되지 않도록 0으로 클리핑
        df["RUL_sec"] = (failure_time_sec - df["time_sec"]).clip(lower=0)

        # summary 딕셔너리를 업데이트
        summary[train_id] = df

    return summary

# 중단 조건 만족 여부에 따라 RUL 방식 적용 함수
def label_rul_pipeline(summary, estimation_targets, threshold_r2, interval_sec):
    """
    Args:
        summary (dict): 각 train_id에 대한 Operation DataFrame을 담은 딕셔너리.
        estimation_targets (dict): 중단 조건 불만족으로 회귀 예측이 필요한 데이터셋의 딕셔너리
                                  (키: train_id, 값: (front_col, rear_col)).
        threshold_r2 (float): 회귀 R² 임계값.
        interval_sec (int): 각 데이터 포인트 간의 시간 간격(초).

    Returns:
        dict: RUL이 라벨링된 DataFrame이 포함된 업데이트된 summary 딕셔너리.
    """
    for train_id, df in summary.items():
        # 중단 조건이 충족되었는지 확인
        if check_stop_condition_met(df):
            # 중단 조건이 충족되었다면, 중단 시점 기반으로 RUL을 라벨링
            updated = label_rul_by_stopping({train_id: df}, interval_sec=interval_sec)
            summary[train_id] = updated[train_id]
            print(f"[{train_id}] 조건 기반 라벨링 완료.")
        # 중단 조건이 충족되지 않았고, 회귀 예측 대상에 포함되는 경우
        elif train_id in estimation_targets:
            # 회귀 기반으로 RUL을 추정하고 라벨링
            updated = label_rul_by_estimation({train_id: df}, {train_id: estimation_targets[train_id]}, threshold_r2, interval_sec)
            summary[train_id] = updated[train_id]
            print(f"[{train_id}] 회귀 기반 라벨링 완료.")
        else:
            # 중단 조건도 없고 회귀 대상도 아닌 경우 라벨링을 생략
            print(f"[{train_id}] 회귀 대상 정보가 없어 라벨링 생략.")

    return summary

# Slope 계산
# slope용 데이터 필터링 함수
def get_train_folders(train_root):
    """
    Args:
        train_root (str): train 데이터셋의 루트 경로.

    Returns:
        list: 정렬된 train 폴더 이름 목록.
    """
    all_folders = os.listdir(train_root)
    # 'train'으로 시작하고 실제 디렉토리인 폴더만 필터링
    train_folders = [f for f in all_folders if f.lower().startswith("train") and os.path.isdir(os.path.join(train_root, f))]
    return sorted(train_folders) # 정렬된 목록 반환

# slope 계산 함수
def compute_slope_feature(df, column_name, N, interval_sec):
    """
    Args:
        df (pd.DataFrame): Operation 데이터가 포함된 DataFrame.
        column_name (str): slope를 계산할 대상 컬럼 이름.
        N (int): slope 계산에 사용할 이전 데이터 포인트 수 (주기).
        interval_sec (int): 각 데이터 포인트 간의 시간 간격(초).

    Returns:
        pd.DataFrame: slope 컬럼이 추가된 DataFrame.
    """
    slope_col = f"{column_name}_Slope_{N}cycle" # 새로운 slope 컬럼 이름
    slopes = [np.nan] * len(df) # 초기값은 NaN으로 채움

    # N번째 데이터 포인트부터 slope를 계산
    for i in range(N, len(df)):
        delta_y = df[column_name].iloc[i] - df[column_name].iloc[i - N] # N 주기 동안의 Y 변화량
        # 시간(X) 변화량: N * interval_sec (실제 time_sec 컬럼 사용)
        delta_x = df["time_sec"].iloc[i] - df["time_sec"].iloc[i - N]
        if delta_x == 0: # 0으로 나누는 것을 방지
            slopes[i] = np.nan
        else:
            slopes[i] = delta_y / delta_x
    df[slope_col] = slopes # 계산된 slope를 새로운 컬럼으로 추가
    return df

# combined slope 계산 함수
def compute_combined_slope(df, temp_col, torque_col, N):
    """
    Args:
        df (pd.DataFrame): slope 피처가 이미 계산된 DataFrame.
        temp_col (str): 온도 컬럼 이름 (예: 'TC SP Front[℃]' 또는 'TC SP Rear[℃]').
        torque_col (str): 토크 컬럼 이름 (예: 'Torque[Nm]').
        N (int): slope 계산에 사용된 주기.

    Returns:
        pd.DataFrame: combined slope 컬럼이 추가된 DataFrame.
    """
    temp_slope_col = f"{temp_col}_Slope_{N}cycle"
    torque_slope_col = f"{torque_col}_Slope_{N}cycle"
    combined_col = f"Combined_Slope_{N}cycle"

    # 필요한 slope 컬럼이 데이터프레임에 있는지 확인
    if temp_slope_col in df.columns and torque_slope_col in df.columns:
        df[combined_col] = df[temp_slope_col] * df[torque_slope_col] # 두 slope 값을 곱하여 새로운 컬럼 생성
    return df

# N=3, N=5에 따른 slope 값 계산 함수
def apply_slope_features(summary, estimation_targets, N_list, interval_sec):
    """
    Args:
        summary (dict): 각 train_id에 대한 Operation DataFrame을 담은 딕셔너리.
        estimation_targets (dict): 각 train_id에 대한 온도 컬럼 이름 튜플 딕셔너리.
        N_list (list): slope를 계산할 N 값 목록 (예: [3, 5]).
        interval_sec (int): 각 데이터 포인트 간의 시간 간격(초).

    Returns:
        dict: slope 피처가 추가된 DataFrame이 포함된 업데이트된 summary 딕셔너리.
    """
    for train_id, df in summary.items():
        # 각 train ID에 해당하는 전면/후면 온도 컬럼 이름을 가져옴.
        # get()을 사용하여 estimation_targets에 해당 train_id가 없을 경우 기본값을 사용.
        front_col, rear_col = estimation_targets.get(train_id, ("TC SP Front[℃]", "TC SP Rear[℃]"))
        df = df.copy() # 원본 DataFrame을 변경하지 않기 위해 복사

        # N_list의 각 N 값에 대해 slope 피처를 계산
        for N in N_list:
            df = compute_slope_feature(df, "Torque[Nm]", N, interval_sec)
            df = compute_slope_feature(df, front_col, N, interval_sec)
            df = compute_slope_feature(df, rear_col, N, interval_sec)
            df = compute_combined_slope(df, front_col, "Torque[Nm]", N)
            df = compute_combined_slope(df, rear_col, "Torque[Nm]", N)

        # slope 계산으로 인해 생긴 NaN(첫 N개 행)을 제거하고 인덱스를 재설정
        df = df.dropna().reset_index(drop=True)
        summary[train_id] = df # 업데이트된 DataFrame을 summary 딕셔너리에 저장
    return summary

# 데이터 확장 (Vibration Data와의 결합을 위한)
def expand_summary_to_windows(summary_df, window_size, overlap, fixed_total_samples):
    """
    Args:
        summary_df (pd.DataFrame): 모든 train_id의 Operation 데이터가 합쳐진 DataFrame.
        window_size (int): 각 윈도우의 샘플 수.
        overlap (float): 윈도우 간의 오버랩 비율 (0.0 ~ 1.0).
        fixed_total_samples (int): 각 TDMS 파일에 포함된 총 진동 샘플 수 (고정된 값).

    Returns:
        pd.DataFrame: 윈도우 인덱스와 함께 확장된 DataFrame.
    """
    step = int(window_size * (1 - overlap)) # 윈도우 이동 간격
    # 총 샘플 수, 윈도우 크기, 스텝을 고려하여 생성될 윈도우의 개수를 계산
    num_windows = (fixed_total_samples - window_size) // step + 1
    expanded_rows = [] # 확장된 행들을 저장할 리스트

    # summary_df의 각 행(즉, 각 TDMS 파일에 대한 operation 데이터)을 순회
    for idx, row in summary_df.iterrows():
        file_name = row.get("file_name", None)
        folder = row.get("folder", None)

        # 필수 컬럼이 있는지 확인
        if file_name is None:
            raise ValueError("summary_df에 'file_name' 컬럼이 없습니다.")
        if folder is None:
            raise ValueError("summary_df에 'folder' 컬럼이 없습니다.")

        # 현재 operation 데이터에 대해 계산된 윈도우 수만큼 행을 복제하고 window_index를 추가
        for w in range(num_windows):
            new_row = row.copy() # 원본 행 복사
            new_row["window_index"] = w # 윈도우 인덱스 추가
            new_row["file_name"] = file_name # 파일 이름 (이름 충돌 방지를 위해 명시적 추가)
            new_row["folder"] = folder # 폴더 이름 (이름 충돌 방지를 위해 명시적 추가)
            expanded_rows.append(new_row)

    # 확장된 행들로 새로운 DataFrame을 생성하여 반환
    expanded_df = pd.DataFrame(expanded_rows)
    return expanded_df

# 전체 데이터 처리 파이프라인
def full_pipeline_with_slope_and_rul(train_root, interval_sec, threshold_r2, N_list):
    """
    Args:
        train_root (str): TDMS 파일이 있는 루트 디렉토리 경로.
        interval_sec (int): 각 데이터 포인트 간의 시간 간격(초).
        threshold_r2 (float): RUL 회귀 R² 임계값.
        N_list (list): slope 계산에 사용할 N 값 목록 (주기).

    Returns:
        dict: 모든 처리 및 피처 엔지니어링이 완료된 DataFrame을 포함하는 딕셔너리.
    """
    # 1. 트레인 폴더 목록을 가져옴
    train_folders = get_train_folders(train_root)
    # RUL 추정에 사용할 기본 타겟 컬럼을 설정
    targets = {folder: ("TC SP Front[℃]", "TC SP Rear[℃]") for folder in train_folders}

    # 2. TDMS 파일에서 Operation 데이터를 로드하여 요약
    summary = load_summary_from_tdms(train_root, train_folders)

    # 3. 중단 조건이 충족되지 않은 데이터셋을 식별하여 회귀 기반 RUL 추정 대상 목록을 만듦
    estimation_targets = {
        train_id: targets[train_id]
        for train_id, df in summary.items()
        if not check_stop_condition_met(df)
    }

    # 4. RUL 라벨링 파이프라인을 실행 (조건 기반 또는 회귀 기반)
    summary = label_rul_pipeline(summary, estimation_targets, threshold_r2=threshold_r2, interval_sec=interval_sec)

    # 5. 슬로프(Slope) 관련 피처를 계산하여 추가
    summary = apply_slope_features(summary, targets, N_list=N_list, interval_sec=interval_sec)

    return summary

# 메인 실행 부분
if __name__ == "__main__":
    # 데이터셋의 루트 경로를 설정
    train_root = "/data/Train Set"

    # 전체 데이터 처리 파이프라인을 실행하여 요약 데이터를 얻음
    summary = full_pipeline_with_slope_and_rul(train_root,
        interval_sec=HYPERPARAMETERS["interval_sec"],
        threshold_r2=HYPERPARAMETERS["threshold_r2"],
        N_list=HYPERPARAMETERS["N_list"])

    # summary는 딕셔너리 형태이므로, 모든 train 데이터를 하나의 DataFrame으로 병합
    summary_df = pd.concat(
        [df.assign(folder=train_id) for train_id, df in summary.items()],
        ignore_index=True
    )

    # Vibration 데이터와의 결합을 위해 Operation 데이터를 윈도우 단위로 확장
    # 이 과정에서 각 operation 데이터 포인트가 여러 개의 "윈도우"에 매핑됨
    summary_expanded = expand_summary_to_windows(
        summary_df,
        window_size=HYPERPARAMETERS["window_size"],
        overlap=HYPERPARAMETERS["overlap"],
        fixed_total_samples=HYPERPARAMETERS["fixed_total_samples"]
    )

    print("\n최종 확장된 Operation Summary DataFrame (상위 5행)")
    print(summary_expanded.head())

    print("\n확장된 DataFrame 정보")
    summary_expanded.info()

    print("\n확장된 DataFrame 기술 통계")
    print(summary_expanded.describe())

    print("\nRUL_sec 컬럼 통계")
    print(summary_expanded['RUL_sec'].describe())

### vibration

In [ ]:
# 고장 주파수 정의 (단위: Hz) - 베어링 고장 시 나타나는 특정한 주파수 성분들
FAULT_FREQUENCIES = {
    "BPFI": 140, # Ball Pass Frequency, Inner Race (내륜 고장 주파수)
    "BPFO": 93,  # Ball Pass Frequency, Outer Race (외륜 고장 주파수)
    "BSF": 78,   # Ball Spin Frequency (볼 스핀 주파수)
    "Cage": 6.7  # Fundamental Train Frequency (케이지 고장 주파수)
}

HYPERPARAMETERS = {
    "window_size": 25600,
    "overlap": 0.5,
    "wavelet": 'db4',
    "level": 3,
    "top_k": 10,
    "bandwidth": 5,
    "duration_sec": 10,
}

# 슬라이딩 윈도우 생성 함수
def sliding_window(data, window_size, overlap):
    """
    Args:
        data (np.array): 입력 시계열 데이터 (예: (샘플 수, 채널 수) 형태의 진동 데이터).
        window_size (int): 각 윈도우의 샘플 수.
        overlap (float): 윈도우 간의 오버랩(겹치는) 비율 (0.0 ~ 1.0).

    Returns:
        np.array: 슬라이딩 윈도우로 분할된 데이터 배열 (윈도우 수, window_size, 채널 수).
    """
    step = int(window_size * (1 - overlap)) # 윈도우가 다음 위치로 이동할 간격
    return np.array([
        data[start:start + window_size] # 시작 인덱스부터 window_size만큼 데이터를 잘라 윈도우 생성
        for start in range(0, len(data) - window_size + 1, step) # 전체 데이터를 step 간격으로 순회
    ])  # 반환되는 배열의 형태: (윈도우 수, window_size, 채널 수)

# WPT+FFT 특징 추출 함수
def extract_wpt_fft_features(signal, wavelet, level, top_k):
    """
    Args:
        signal (np.array): 단일 채널의 진동 신호.
        wavelet (str): 사용할 웨이블릿 종류 (예: 'db4').
        level (int): WPT 분해 레벨.
        top_k (int): 각 웨이블릿 노드(주파수 대역)에서 FFT 진폭 상위 K개를 추출.

    Returns:
        np.array: 추출된 특징들의 1D 배열 (노드 수 × top_k,).
    """
    # 웨이블릿 패킷 변환 객체 생성
    wp = pywt.WaveletPacket(data=signal, wavelet=wavelet, mode='symmetric', maxlevel=level)
    # 지정된 레벨의 모든 노드(주파수 대역) 경로를 가져옴
    nodes = [node.path for node in wp.get_level(level, 'freq')]
    features = []

    for node in nodes:
        coeffs = wp[node].data # 각 노드의 웨이블릿 계수(데이터)
        fft_vals = np.abs(rfft(coeffs)) # 실수 푸리에 변환 (FFT)의 절대값 (진폭 스펙트럼)
        top_features = np.sort(fft_vals)[-top_k:] # FFT 진폭 중 상위 K개 추출
        features.extend(top_features) # 추출된 특징들을 리스트에 추가
    return np.array(features) # 모든 특징들을 하나의 넘파이 배열로 반환

# TDMS 파일 경로로부터 진동 데이터를 읽어, 윈도우별 WPT+FFT 특징 추출 함수
def extract_vibration_array_with_features(file_path, window_size, overlap, wavelet, level, top_k):
    """
    Args:
        file_path (str): TDMS 파일의 전체 경로.
        window_size (int): 슬라이딩 윈도우의 샘플 수.
        overlap (float): 윈도우 간의 오버랩 비율.
        wavelet (str): WPT에 사용할 웨이블릿 종류.
        level (int): WPT 분해 레벨.
        top_k (int): WPT+FFT에서 추출할 상위 K개 특징.

    Returns:
        np.array: 각 윈도우에 대한 모든 채널의 추출된 특징 배열 (윈도우 수, 채널 수 * 특징 수).
    """
    vib_data, _ = load_tdms_file(file_path) # TDMS 파일 로드 (진동 데이터만 필요)
    channels = ["CH1", "CH2", "CH3", "CH4"] # 분석할 채널 목록
    # 실제 vib_data에 존재하는 채널의 데이터만 추출하여 리스트로 만듦
    vib_arrays = [vib_data[ch] for ch in channels if ch in vib_data]
    vib_matrix = np.vstack(vib_arrays).T # 각 채널 데이터를 수직으로 쌓아 (샘플 수, 4) 형태의 행렬로 변환

    # 진동 행렬에 대해 슬라이딩 윈도우를 적용
    windows = sliding_window(vib_matrix, window_size=window_size, overlap=overlap) # (윈도우 수, window_size, 4)
    all_features = [] # 모든 윈도우의 특징들을 저장할 리스트

    # 각 윈도우를 순회하며 특징 추출
    for window in windows:
        window_features = [] # 현재 윈도우의 모든 채널 특징을 저장할 리스트
        # 각 채널을 순회하며 특징 추출
        for ch_idx in range(window.shape[1]): # window.shape[1]은 채널 수 (4)
            signal = window[:, ch_idx] # 현재 채널의 신호 추출
            feat = extract_wpt_fft_features(signal, wavelet=wavelet, level=level, top_k=top_k) # WPT+FFT 특징 추출
            window_features.extend(feat) # 추출된 특징을 현재 윈도우 특징 리스트에 추가
        all_features.append(window_features) # 현재 윈도우의 모든 채널 특징을 전체 특징 리스트에 추가

    return np.array(all_features) # 모든 윈도우에 대한 특징들을 넘파이 배열로 반환 (윈도우 수, 채널 수 * 특징 수)

# 고장 주파수 기반 FFT 특징 추출
def extract_fault_frequency_features(signal, Fs, bandwidth, fault_freqs=FAULT_FREQUENCIES):
    """
    Args:
        signal (np.array): 단일 채널의 진동 신호.
        Fs (float): 샘플링 주파수 (Hz).
        fault_freqs (dict): 고장 주파수 이름과 중심 주파수를 매핑한 딕셔너리.
        bandwidth (int): 각 고장 주파수 대역의 폭 (중심 주파수 ± bandwidth).

    Returns:
        dict: 각 고장 주파수 특징 이름(예: "BPFI_Energy", "BPFO_Peak_Amplitude")을 키로 하는 딕셔너리.
    """
    N = len(signal) # 신호의 샘플 수
    # FFT 주파수 축 생성 (양수 주파수만)
    freqs = np.fft.fftfreq(N, d=1/Fs)[:N//2]
    # FFT 진폭 스펙트럼 계산 (양수 주파수만)
    fft_magnitude = np.abs(np.fft.fft(signal))[:N//2]

    features = {}

    for fault_name, f_center in fault_freqs.items():
        lower = f_center - bandwidth # 주파수 대역의 하한
        upper = f_center + bandwidth # 주파수 대역의 상한
        band_mask = (freqs >= lower) & (freqs <= upper) # 해당 주파수 대역에 속하는 인덱스 마스크
        band_energy = np.sum(fft_magnitude[band_mask] ** 2) # 해당 대역 내 FFT 진폭의 제곱 합 (에너지)

        if np.any(band_mask): # 해당 대역에 주파수 성분이 있다면
            peak_idx_in_band = np.argmax(fft_magnitude[band_mask]) # 해당 대역 내에서 피크 진폭의 상대 인덱스
            band_mags = fft_magnitude[band_mask]
            peak_amp = band_mags[peak_idx_in_band] # 피크 진폭 값
        else: # 해당 대역에 주파수 성분이 없다면
            peak_amp = 0.0

        features[f"{fault_name}_Energy"] = band_energy # 에너지 특징 추가
        features[f"{fault_name}_Peak_Amplitude"] = peak_amp # 피크 진폭 특징 추가

    return features

# TDMS 파일 단일 처리, 멀티 채널 고장 주파수 특징 추출
def extract_fault_features_from_tdms(vib_data, duration_sec, bandwidth):
    """
    Args:
        vib_data (dict): TDMS에서 읽은 진동 데이터 딕셔너리 (예: {"CH1": array, ...}).
        duration_sec (int): 진동 시퀀스의 총 길이(초). (샘플링 주파수 Fs 계산에 사용됨)
        bandwidth (int): 각 고장 주파수 대역의 폭.

    Returns:
        dict: 모든 채널의 고장 주파수 특징을 담은 딕셔너리 (예: "CH1_BPFI_Energy").
    """
    features = {}
    channels = ["CH1", "CH2", "CH3", "CH4"] # 처리할 채널 목록

    # 샘플 수와 duration_sec을 이용하여 샘플링 주파수(Fs) 계산
    sample_counts = [len(vib_data[ch]) for ch in channels if ch in vib_data]
    if not sample_counts: # 유효한 진동 채널이 없으면 오류 발생
        raise ValueError("No valid vibration channels found.")
    N = sample_counts[0] # 첫 번째 채널의 샘플 수를 기준으로 N 설정 (모든 채널의 샘플 수가 동일하다고 가정)
    Fs = N / duration_sec # 샘플링 주파수 (Hz)

    for ch in channels:
        if ch not in vib_data: # 현재 채널이 vib_data에 없으면 건너뜀
            continue
        signal = vib_data[ch] # 현재 채널의 신호
        fault_feats = extract_fault_frequency_features(signal, Fs, bandwidth=bandwidth) # 단일 채널 고장 주파수 특징 추출
        for k, v in fault_feats.items():
            features[f"{ch}_{k}"] = v # 채널 이름 접두사를 붙여 특징 딕셔너리에 추가

    return features

# WPT+FFT 윈도우별 특징 추출 및 파일 단위 요약
def process_all_train_folders(base_path, folder_names, window_size, overlap, wavelet, level, top_k):
    """
    Args:
        base_path (str): 트레인 데이터셋의 루트 경로.
        folder_names (list): 처리할 트레인 폴더 이름 목록.

    Returns:
        pd.DataFrame: 각 윈도우의 특징과 메타데이터를 담은 DataFrame.
    """
    all_feature_rows = [] # 모든 윈도우 특징들을 저장할 리스트

    for folder_name in tqdm(folder_names, desc="Processing WPT+FFT features"): # tqdm으로 진행률 표시
        folder_path = os.path.join(base_path, folder_name) # 현재 폴더 경로
        tdms_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.tdms')]) # TDMS 파일 목록 정렬

        for tdms_file in tdms_files:
            file_path = os.path.join(folder_path, tdms_file) # 현재 TDMS 파일 전체 경로
            # 파일에서 윈도우별 WPT+FFT 특징 추출
            features = extract_vibration_array_with_features(
                file_path,
                window_size=window_size,
                overlap=overlap,
                wavelet=wavelet,
                level=level,
                top_k=top_k
            )

            for i, feature in enumerate(features): # 각 윈도우의 특징을 순회
                # 윈도우별 특징과 메타데이터를 딕셔너리로 만들어 리스트에 추가
                all_feature_rows.append({
                    'file_name': tdms_file,
                    'window_index': i,
                    'features': feature, # 추출된 특징 배열
                    'folder': folder_name
                })

    return pd.DataFrame(all_feature_rows) # 모든 윈도우 특징들을 DataFrame으로 변환

# 모든 train 폴더에 고장 주파수 특징 추출 적용
def extract_fault_features_all(train_root, bandwidth, target_folders=None):
    """
    Args:
        train_root (str): 트레인 데이터셋의 루트 경로.
        target_folders (list, optional): 처리할 특정 폴더 목록. None이면 train_root의 모든 하위 폴더를 처리.
        bandwidth (int): 고장 주파수 대역의 폭.

    Returns:
        pd.DataFrame: 각 TDMS 파일의 고장 주파수 특징을 담은 DataFrame.
    """
    if target_folders is None:
        target_folders = sorted(os.listdir(train_root)) # 모든 하위 폴더를 가져옴

    all_records = [] # 모든 파일의 특징들을 저장할 리스트

    for folder in tqdm(target_folders, desc="Extracting fault features"): # tqdm으로 진행률 표시
        folder_path = os.path.join(train_root, folder)
        tdms_files = sorted([f for f in os.listdir(folder_path) if f.endswith(".tdms")])

        for tdms_file in tdms_files:
            file_path = os.path.join(folder_path, tdms_file)
            try:
                vib_data, _ = load_tdms_file(file_path) # TDMS 파일 로드
                features = extract_fault_features_from_tdms(
                    vib_data,
                    duration_sec=HYPERPARAMETERS["duration_sec"],
                    bandwidth=bandwidth
                ) # 고장 주파수 특징 추출
                features["folder"] = folder # 폴더 이름 추가
                features["file_name"] = tdms_file.replace('.tdms', '') # .tdms 확장자 제거
                all_records.append(features) # 특징 딕셔너리를 리스트에 추가
            except Exception as e:
                print(f"Failed to process {file_path}: {e}") # 파일 처리 실패 시 오류 메시지 출력

    df = pd.DataFrame(all_records) # 모든 특징 딕셔너리들을 DataFrame으로 변환
    return df

# 윈도우별 특징을 파일 단위 summary로 변환(평균, 표준편차)
def summarize_window_features_compact(features_df):
    """
    Args:
        features_df (pd.DataFrame): 윈도우별 WPT+FFT 특징을 담은 DataFrame (process_all_train_folders의 결과).

    Returns:
        pd.DataFrame: 파일별로 WPT+FFT 특징의 평균과 표준편차를 요약한 DataFrame.
    """
    summary_records = []

    # 'folder'와 'file_name' 기준으로 그룹화
    for (folder, file_name), group in features_df.groupby(['folder', 'file_name']):
        feature_array = np.stack(group['features'].values) # 그룹 내 모든 'features' 배열을 하나의 넘파이 배열로 쌓음 (윈도우 수, 피처 수)
        feature_mean = np.mean(feature_array, axis=0) # 각 피처의 평균 계산
        feature_std = np.std(feature_array, axis=0) # 각 피처의 표준편차 계산

        record = {
            'folder': folder,
            'file_name': file_name,
            'WPTFFT_Feature_mean': feature_mean, # 평균 특징을 컬럼으로 추가
            'WPTFFT_Feature_std': feature_std,   # 표준편차 특징을 컬럼으로 추가
        }
        summary_records.append(record)

    return pd.DataFrame(summary_records) # 요약된 레코드들을 DataFrame으로 변환

# 설정값
base_path = "/data/Train Set"
folder_names = [f"Train{i}" for i in range(1, 9)] # Train1 ~ Train8 폴더

# WPT+FFT 윈도우 특징 추출 수행
features_df = process_all_train_folders(
    base_path,
    folder_names,
    window_size=HYPERPARAMETERS["window_size"],
    overlap=HYPERPARAMETERS["overlap"],
    wavelet=HYPERPARAMETERS["wavelet"],
    level=HYPERPARAMETERS["level"],
    top_k=HYPERPARAMETERS["top_k"]
)
# 파일 이름에서 '.tdms' 확장자를 제거하여 이후 병합을 용이하게 함
features_df['file_name'] = features_df['file_name'].str.replace(r'\.tdms$', '', regex=True)

# WPT+FFT 윈도우 특징들을 파일 단위로 요약 (평균, 표준편차)
wptfft_summary_df = summarize_window_features_compact(features_df)

# 고장 주파수 FFT 특징 추출
fault_summary_df = extract_fault_features_all(
    base_path,
    bandwidth=HYPERPARAMETERS["bandwidth"],
    target_folders=folder_names
)

# 추출된 두 DataFrame의 컬럼 목록 출력하여 확인
print("\nwptfft_summary_df columns:", wptfft_summary_df.columns.tolist())
print("fault_summary_df columns:", fault_summary_df.columns.tolist())

# 두 summary DataFrame 병합
# 'folder'와 'file_name'을 키로 사용하여 두 DataFrame을 내부 조인(inner join)으로 병합
summary_df = pd.merge(wptfft_summary_df, fault_summary_df, on=['folder', 'file_name'], how='inner')

# 병합된 결과에 대한 정보 출력
print(f"\n요약된 파일 수 (병합 후): {len(summary_df)}")
print(f"총 윈도우 수 (WPT+FFT 추출 시): {len(features_df)}")

# 최종 병합된 DataFrame의 상위 5행과 정보 출력
print("\n최종 병합된 특징 Summary DataFrame (상위 5행)")
print(summary_df.head())
print("\n병합된 DataFrame 정보")
summary_df.info()
print("\n병합된 DataFrame 기술 통계")
print(summary_df.describe())

### 병합 및 데이터 로드

In [ ]:
print("summary_expanded columns:", summary_expanded.columns.tolist())
print("features_df columns:", features_df.columns.tolist())

# 데이터프레임 병합
# summary_expanded와 features_df를 'folder', 'file_name', 'window_index' 기준으로 병합
merged_df = pd.merge(
    summary_expanded, # Operation 데이터(RUL, 슬로프 특징 포함)를 윈도우 단위로 확장한 DataFrame
    features_df,      # Vibration 데이터에서 윈도우별로 추출된 WPT+FFT 특징 DataFrame
    on=['folder', 'file_name', 'window_index'], # 병합 기준이 되는 키(컬럼)들
    how='inner'       # 내부 조인 (두 DataFrame에 모두 존재하는 행만 유지)
)

print("병합된 데이터 개수:", len(merged_df)) # 병합 후 총 데이터(윈도우) 개수 출력
print(merged_df.columns) # 병합된 DataFrame의 모든 컬럼 이름 출력

# 설정값
data_dir = "Train Set"       # 데이터셋의 기본 디렉토리
window_size = 25600          # 진동 데이터 윈도우의 샘플 수
overlap = 0.5                # 윈도우 간의 오버랩 비율 (50%)
wavelet = 'db4'              # 웨이블릿 변환에 사용할 웨이블릿 종류
level = 3                    # 웨이블릿 패킷 변환의 분해 레벨
top_k = 10                   # WPT+FFT 특징으로 추출할 상위 K개 주파수 성분
sampling_rate = 25600        # 샘플링 주파수 (Hz)
step = int(window_size * (1 - overlap)) # 윈도우 이동 간격 계산

# 특징(X) 및 라벨(y) 데이터 준비
X = np.stack(merged_df['features'].values) # 리스트 형태의 특징 배열들을 하나의 넘파이 배열로 스택 (N, feature_dim)
X = X[..., np.newaxis] # CNN/LSTM 모델의 입력 형태에 맞게 마지막에 채널 차원 추가 (N, feature_dim, 1)

y = merged_df['RUL_sec'].values # RUL 라벨 추출 (N,)

# 데이터 스케일링
# Min-Max 스케일링을 사용하여 데이터를 0과 1 사이로 정규화
X_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

# X(특징) 스케일링
# MinMaxScaler는 2D 배열을 입력으로 기대하므로, X를 일시적으로 2D로 reshape.
samples, feat_dim, channels = X.shape
X_reshaped = X.reshape(samples, feat_dim * channels) # (samples, feature_dim * channels)
X_scaled = X_scaler.fit_transform(X_reshaped)

# y(RUL 라벨) 스케일링
# y도 2D 배열이어야 하므로 reshape(-1, 1)을 사용하여 2D로 reshape.
y_scaled = y_scaler.fit_transform(y.reshape(-1,1))


X_train_final = X_scaled.reshape(samples, feat_dim, channels)  # (samples, feature_dim, channels)
y_train_final = y_scaled

# 최종 데이터 형태 출력하여 확인
print(f"X_train_final shape: {X_train_final.shape}, y_train_final shape: {y_train_final.shape}")

# 스케일러 저장
# 스케일러를 저장할 디렉토리 경로 정의
scaler_dir = "/data/scalers"
os.makedirs(scaler_dir, exist_ok=True) # 디렉토리가 없으면 생성

# X_scaler와 y_scaler 객체를 지정된 경로에 .pkl 파일로 저장
joblib.dump(X_scaler, os.path.join(scaler_dir, "X_scaler.pkl"))
joblib.dump(y_scaler, os.path.join(scaler_dir, "y_scaler.pkl"))
print(f"스케일러가 '{scaler_dir}' 에 성공적으로 저장되었습니다.")

## 학습

### simple CNN-LSTM

In [ ]:
# Conv1D와 LSTM layer 결합
def simplified_conv1d_lstm_model_v1(input_shape):
    """
    Args:
        input_shape (tuple): 모델 입력 데이터의 형태 (예: (특징 수, 채널 수)).

    Returns:
        tf.keras.models.Model: 컴파일되지 않은 Keras 모델 객체.
    """
    input_layer = Input(shape=input_shape) # 모델의 입력 레이어 정의

    # Conv1D 레이어: 시계열 데이터에서 지역적인 특징을 추출.
    x = Conv1D(filters=8, kernel_size=3, activation='relu', kernel_regularizer=regularizers.l2(0.005), padding='causal')(input_layer)
    x = MaxPooling1D(pool_size=2)(x)
    x = Dropout(0.2)(x)

    # LSTM 레이어: 시계열 데이터의 의존성 학습.
    lstm_out = LSTM(16, return_sequences=False, kernel_regularizer=regularizers.l2(0.005))(x)
    dropout_lstm = Dropout(0.3)(lstm_out)

    # Dense 레이어: 최종 예측을 수행.
    output = Dense(1, activation='linear')(dropout_lstm)

    model = Model(inputs=input_layer, outputs=output) # 모델 객체 생성
    return model

# 모델 생성 및 컴파일
input_shape = X_train_final.shape[1:]

model = simplified_conv1d_lstm_model_v1(input_shape)

# 모델 컴파일
custom_adam_optimizer_1 = Adam(learning_rate=0.0001)
model.compile(optimizer=custom_adam_optimizer_1, loss='mse')
model.summary()

# 콜백 설정
early_stop = EarlyStopping(
    patience=7,
    restore_best_weights=True,
    monitor='val_loss'
)
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=7,
    min_lr=0.00001,
    verbose=1
)

# 모델 학습
history = model.fit(
    X_train_final, y_train_final,
    batch_size=32,
    epochs=100,
    validation_split=0.1,
    callbacks=[early_stop, reduce_lr]
)

# 모델 저장
save_path = "/data/model"
os.makedirs(save_path, exist_ok=True)

model.save(os.path.join(save_path, "simple_cnn_lstm_model.h5"))
print("모델이 'simple_cnn_lstm_model.h5'에 성공적으로 저장되었습니다.")

### Validation

In [ ]:
base_path = "/data/Validation Set"
folder_names = [f"Validation{i}" for i in range(1, 7)]
targets = {folder: ("TC SP Front[℃]", "TC SP Rear[℃]") for folder in folder_names}

# Operation 데이터(슬로프 특징) 추출 및 전처리
summary = load_summary_from_tdms(base_path, folder_names)
operation_summary = apply_slope_features(summary, targets, N_list=[3, 5], interval_sec=600)

# summary는 dict이므로 하나의 DataFrame으로 병합
summary_df = pd.concat(
    [df.assign(folder=train_id) for train_id, df in operation_summary.items()],
    ignore_index=True
)

# operation data를 윈도우 단위로 확장
summary_expanded = expand_summary_to_windows(
    summary_df,
    window_size=HYPERPARAMETERS["window_size"],
    overlap=HYPERPARAMETERS["overlap"],
    fixed_total_samples=256000
    )

# vibration data WPT+FFT 윈도우 특징 추출 수행
features_df = process_all_train_folders(
    base_path,
    folder_names,
    window_size=HYPERPARAMETERS["window_size"],
    overlap=HYPERPARAMETERS["overlap"],
    wavelet=HYPERPARAMETERS["wavelet"],
    level=HYPERPARAMETERS["level"],
    top_k=HYPERPARAMETERS["top_k"]
)
# .tdms 확장자 제거
features_df['file_name'] = features_df['file_name'].str.replace(r'\.tdms$', '', regex=True)

# 윈도우 별로 추출된 WPT+FFT 특징들을 파일 단위로 요약
wptfft_summary_df = summarize_window_features_compact(features_df)
fault_summary_df = extract_fault_features_all(
    base_path,
    bandwidth=HYPERPARAMETERS["bandwidth"],
    target_folders=folder_names
    )

# 두 summary의 dataframe 병합
summary_df = pd.merge(wptfft_summary_df, fault_summary_df, on=['folder', 'file_name'], how='inner')

print(f"요약된 파일 수: {len(summary_df)}")
print(f"총 윈도우 수: {len(features_df)}")

# 최종 데이터셋 병합
merged_df = pd.merge(
    summary_expanded,
    features_df,
    on=['folder', 'file_name', 'window_index'],
    how='inner'  # 윈도우 단위로 완전히 겹치는 데이터만 남음
)

print("병합된 데이터 개수:", len(merged_df))
print(merged_df.columns)

In [ ]:
# 모델 로드
model_path = "/data/model/simple_cnn_lstm_model.h5"
model = load_model(model_path, compile=False)
print("모델 로드 완료:", model_path)

# 스케일러 로드
X_scaler = joblib.load("/data/scalers/X_scaler.pkl")
y_scaler = joblib.load("/data/scalers/y_scaler.pkl")

# 특징 배열 준비
X_val = np.stack(merged_df['features'].values)  # (samples, features)
X_val = X_val[..., np.newaxis]  # (samples, features, 1)

# reshape 후 스케일링
samples_val, feat_dim, channels = X_val.shape
X_val_reshaped = X_val.reshape(samples_val, feat_dim * channels)  # 2D (samples, features)

X_val_scaled = X_scaler.transform(X_val_reshaped)

# 스케일된 데이터를 원래 형태로 reshape
X_val_final = X_val_scaled.reshape(samples_val, feat_dim, channels)  # 3D 입력

# 예측
y_val_pred_scaled = model.predict(X_val_final, verbose=1)

# 역스케일링
y_val_pred = y_scaler.inverse_transform(y_val_pred_scaled)

# 결과 병합
merged_df['RUL_pred_sec'] = y_val_pred .flatten()

# 예측 결과 확인
print(merged_df[['folder', 'file_name', 'window_index', 'RUL_pred_sec']].head(6))

In [ ]:
# 파라미터 정의 (기존 슬라이딩 윈도우 기준)
window_size = 25600           # 윈도우 샘플 수
overlap = 0.5                # 오버랩 비율
step = int(window_size * (1 - overlap))  # 윈도우 이동 간격 (샘플 수)
sampling_rate = 25600         # 샘플링 주파수 (Hz), 예: 25600 샘플/초

data_length_samples_dict = {
    'Validation1': 38400000,
    'Validation2': 12800000,
    'Validation3': 9728000,
    'Validation4': 7936000,
    'Validation5': 5888000,
    'Validation6': 5376000,
}


# merged_df의 마지막 window_index를 구한다 (폴더별)
last_window_indices = merged_df.groupby('folder')['window_index'].max()

offset_dict = {}

for folder, last_idx in last_window_indices.items():
    data_length = data_length_samples_dict[folder]  # 폴더별 실제 raw 데이터 길이 (샘플 수)

    last_window_start_sample = last_idx * step
    last_window_end_sample = last_window_start_sample + window_size

    offset_samples = data_length - last_window_end_sample
    offset_samples = max(0, offset_samples)  # 음수 방지

    offset_sec = offset_samples / sampling_rate

    offset_dict[folder] = offset_sec

print(offset_dict)

# 기존 마지막 윈도우별 예측 RUL값을 담은 DataFrame
final_rul_scores = (
    merged_df.groupby('folder')
    .apply(lambda df: df.sort_values('window_index').iloc[-1])  # 마지막 윈도우
    .reset_index(drop=True)
)

# offset 값을 열로 추가
final_rul_scores['offset_sec'] = final_rul_scores['folder'].map(offset_dict)

# offset을 더한 보정 RUL 계산
final_rul_scores['RUL_pred_sec_corrected'] = final_rul_scores['RUL_pred_sec'] + final_rul_scores['offset_sec']

# 결과 확인
print(final_rul_scores[['folder', 'RUL_pred_sec', 'offset_sec', 'RUL_pred_sec_corrected']])

import pandas as pd

# offset 더한 보정 RUL 계산 (이미 계산했다고 가정)
final_rul_scores['offset_sec'] = final_rul_scores['folder'].map(offset_dict)
final_rul_scores['RUL_pred_sec_corrected'] = final_rul_scores['RUL_pred_sec'] + final_rul_scores['offset_sec']

# 제출용 데이터프레임 생성
result_df = pd.DataFrame({
    'File': final_rul_scores['folder'],
    'RUL_Score(sec)': final_rul_scores['RUL_pred_sec_corrected']
})

# 엑셀 파일 경로 지정
output_path = "/data/RUL_Score.xlsx"

# 엑셀 저장
result_df.to_excel(output_path, index=False)

print("보정된 RUL 점수 엑셀 저장 완료:", output_path)
print(result_df)
